In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from datetime import datetime
import csv
import pandas as pd
import time


In [2]:
# Spotify API credentials
client_id = 'dfb4f42769e24d8a9870f81c6b660989'
client_secret = 'd76afa6f5ab64a368e4dc4752084f11c'

# Set up authentication using client credentials flow
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

# Playlist ID - Top 50 Polska
playlist_id = '37i9dQZEVXbN6itCcaL3Tt'
# '37i9dQZF1DXcBWIGoYBM5M' - Top Global


In [3]:
# Funkcja zapisująca informacje o utworach do pliku CSV
def save_to_csv(track_info_list, csv_file):
    fieldnames = ['rank', 'name', 'artist', 'date_added', 'popularity']

    with open(csv_file, mode='a', newline='', encoding='utf-8') as file:
        writer = csv.DictWriter(file, fieldnames=fieldnames, delimiter=';')

        # Sprawdź, czy plik CSV jest pusty, a jeśli tak, napisz nagłówki
        if file.tell() == 0:
            writer.writeheader()

        # Zapisz informacje o utworach do pliku CSV
        writer.writerows(track_info_list)

In [4]:
def load_existing_tracks(csv_file_path):
    # Wczytaj istniejące informacje o utworach z pliku CSV
    try:
        existing_tracks = pd.read_csv(csv_file_path, parse_dates=['date_added'], sep=';')
        return existing_tracks
    except FileNotFoundError:
        return None

In [5]:
def retrive_track_info(playlist_id,frequency=None, csv_file_path='data/lololo_tracks.csv'): # USTAW NA TOP TRACKS

    # Pętla, która uruchamia się co godzinę
    # while True:
    try:

        # Pobierz top 50 utworów z playlisty
        top_tracks = sp.playlist_tracks(playlist_id, limit=50)

        # Sprawdź, czy plik CSV istnieje
        existing_tracks = load_existing_tracks(csv_file_path)

        # Przygotuj listę informacji o utworach do zapisu
        track_info_list = []

        # Sprawdź datę dodania pierwszego utworu z playlisty
        first_track_date_added = None
        if top_tracks['items']:
            first_track_date_added = datetime.strptime(top_tracks['items'][0]['added_at'], '%Y-%m-%dT%H:%M:%SZ')

        new_rank_exists = True

        # Sprawdź, czy data dodania utworu istnieje w pliku CSV
        if existing_tracks is not None and not existing_tracks.empty and 'date_added' in existing_tracks:
            last_track_date_added = existing_tracks['date_added'].iloc[-1]
            if first_track_date_added and first_track_date_added.date() == last_track_date_added.date():
                print("Brak nowych utworów do dodania.")
                new_rank_exists = False 
                
        if new_rank_exists:
               
            for idx, track in enumerate(top_tracks['items'], start=1):
                track_info = {
                    'rank': idx,
                    'name': track['track']['name'],
                    'artist': track['track']['artists'][0]['name'],
                    'date_added': datetime.strptime(track['added_at'], '%Y-%m-%dT%H:%M:%SZ'),
                    'popularity': track['track']['popularity']
                }
                track_info_list.append(track_info)
            
            # Zapisz informacje o utworach do pliku CSV
            save_to_csv(track_info_list, csv_file_path)
            
            # Wyświetl informacje o utworach
            for track_info in track_info_list:
                print(f"{track_info['rank']}. {track_info['name']} by {track_info['artist']} (Popularity: {track_info['popularity']})")

    except spotipy.SpotifyException as e:
        print(f"Spotify API error: {e}")
    except Exception as e:
        print(f"Error: {e}")

    # Częstotliwość wykonywania pętli w sekundach
    # time.sleep(frequency)
    return

In [6]:
# Ręczne pobieranie utworów
retrive_track_info(playlist_id,csv_file_path='data/top_tracks.csv')

## Zautomatyzowane pobieranie utworów
# retrive_track_info(playlist_id,86400,csv_file_path='data/top_tracks_automatic.csv')

Brak nowych utworów do dodania.
